# RAG (Retrieval Augmented Generation)

Give extra private / up to date context to LLM when answering questions

- Models are trained with data up to point X in time.
- Most useful applications require up to date information.

## Import dependencies

In [1]:
var Bedrock = require('@langchain/community/llms/bedrock').Bedrock;
var ChatPromptTemplate = require('@langchain/core/prompts').ChatPromptTemplate;

## Instantiate the `model` client

In [2]:
var model = new Bedrock({
    model_id:'amazon.titan-text-express-v1',
    temperature: 1,
    maxTokenCount: 512,
    topP: 0.9,
    verbose: true
});

In [3]:
var input = "What is LCEL ?";

In [ ]:
model.invoke(input).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "What is LCEL ?"
  ]
}
[llm/end] [1:llm:Bedrock] [2.20s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "\nLCEL is a type of cell created by combining two or more cells with specific functionalities, such as a battery cell and a solar cell, to create a hybrid cell that can store energy and generate electricity."
      }
    ]
  ]
}

LCEL is a type of cell created by combining two or more cells with specific functionalities, such as a battery cell and a solar cell, to create a hybrid cell that can store energy and generate electricity.


When asked about `LCEL`, the model gives information regarding hybrid battery cell technology.

- we want to get an answer regarding LangChain [`LCEL`](https://python.langchain.com/v0.1/docs/expression_language/)
- we need to give up to date information regarding this data point as context to the model
- when we give no prompt, the default prompt is similar to the following:


In [8]:
var prompt = ChatPromptTemplate.fromTemplate(`
    What is {input} ?
`);

In [7]:
var input = "LCEL";

## Create a chain

- We create a basic chain with simple prompt and model

In [9]:
var chain = prompt.pipe(model);

## Invoke LLM

- Given input we invoke LLM and get response on what LLM knows about LCEL by default using prompt template

In [ ]:
chain.invoke({input}).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: \n    What is LCEL ?"
  ]
}
[llm/end] [1:llm:Bedrock] [2.59s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "Bot: The LCEL, or lower convective bounding layer, is a layer of the atmosphere that plays a crucial role in the transfer of energy between the Earth's surface and the upper atmosphere. It forms a boundary between the relatively warm and"
      }
    ]
  ]
}
Bot: The LCEL, or lower convective bounding layer, is a layer of the atmosphere that plays a crucial role in the transfer of energy between the Earth's surface and the upper atmosphere. It forms a boundary between the relatively warm and


## Adding context

- We can add context to the query in multiple ways

### 1. Directly injected in prompt

In [11]:
var promptContextHardcoded = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Context: LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).
    Question: {input}
`);

In [14]:
var input = 'What is LCEL ?'

#### Create a chain

- We create a chain with prompt containing hardcoded context, input placeholder and model

In [12]:
var chainContextHardcoded = promptContextHardcoded.pipe(model);

#### Invoke LLM

- Given input we invoke LLM and get response on what LLM knows about LCEL by given extra context with information about LCEL

In [ ]:
chainContextHardcoded.invoke({input}).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: \n    Answer the user question.\n    Context: LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).\n    Question: What is LCEL ?"
  ]
}
[llm/end] [1:llm:Bedrock] [2.30s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "LCEL is a programming language designed to be easy to use and can be used for a variety of tasks, from simple prompts to complex chains."
      }
    ]
  ]
}
LCEL is a programming language designed to be easy to use and can be used for a variety of tasks, from simple prompts to complex chains.


### 2. Passed as a parameter

#### Context documents

- In order to pass in dynamic context we need to be able to provide the data in a format that can be easily processed by framework steps

`@langchain/core/documents` contains the blueprints for additional data that can be injected
`langchain/chains/combine_documents` contains the function to create the generic document pipeable function

In [16]:
var Document = require("@langchain/core/documents").Document;
var createStuffDocumentsChain = require("langchain/chains/combine_documents").createStuffDocumentsChain;

#### Create prompt with contexy placeholder

In [17]:
var promptContexDynamic = ChatPromptTemplate.fromTemplate(`
    Answer the user question.
    Context: {context}
    Question: {input}
`);

#### Create a chain

- We create a chain with prompt containing context, input placeholders and model

In [18]:
var chainContextDynamic;

createStuffDocumentsChain({
    llm: model,
    prompt: promptContexDynamic,
}).then((chain) => chainContextDynamic = chain);

Promise { <pending> }

#### Create documents with additional information

In [21]:
var document1 = new Document({
    pageContent: 'LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).'
});

var document2 = new Document({
    pageContent: 'Password is "OMEGAEPSILON"'
});

#### Invoke LLM

- given input we invoke LLM and get response on what LLM knows about LCEL by given extra context with information about LCEL
- it is injected in invoke call as chain is a `documents` capable chain

In [ ]:
chainContextDynamic.invoke({
    input,
    context: [document1,document2]
}).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: \n    Answer the user question.\n    Context: LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).\n\nPassword is \"OMEGAEPSILON\"\n    Question: What is LCEL ?"
  ]
}
[llm/end] [1:llm:Bedrock] [1.39s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "\nLCEL is LangChain Expression Language ."
      }
    ]
  ]
}

LCEL is LangChain Expression Language .


We can also ask for information from the other document on the same chain

In [23]:
var input = 'What is the password ?';

In [ ]:
chainContextDynamic.invoke({
    input,
    context: [document1,document2]
}).then((response) => console.log(response));

Promise { <pending> }

[llm/start] [1:llm:Bedrock] Entering LLM run with input: {
  "prompts": [
    "Human: \n    Answer the user question.\n    Context: LangChain Expression Language, or LCEL, is a declarative way to easily compose chains together. LCEL was designed from day 1 to support putting prototypes in production, with no code changes, from the simplest “prompt + LLM” chain to the most complex chains (we’ve seen folks successfully run LCEL chains with 100s of steps in production).\n\nPassword is \"OMEGAEPSILON\"\n    Question: What is the password ?"
  ]
}
[llm/end] [1:llm:Bedrock] [1.75s] Exiting LLM run with output: {
  "generations": [
    [
      {
        "text": "The password is \"OMEGAEPSILON\"."
      }
    ]
  ]
}
The password is "OMEGAEPSILON".
